In [1]:
import pandas as pd
import numpy as np
import gc
import os, shutil
from tqdm import tqdm_notebook

In [2]:
NPY_PATH = "/home/saby/Projects/ati/data/data/datasets/Carla/64beam-Data/dynamic/6/_segment_out_npy/0.npy"
DYNAMISM_THRESHOLD = 0.5
assert DYNAMISM_THRESHOLD >= 0.01 and DYNAMISM_THRESHOLD <= 1
OUTPUT_PATH = "/home/saby/Projects/ati/data/data/datasets/Carla/64beam-Data/dynamic/6/_random_angle_" + str(DYNAMISM_THRESHOLD) + "_out_npy"
if not os.path.exists(OUTPUT_PATH):
    os.makedirs(OUTPUT_PATH)
else:
    shutil.rmtree(OUTPUT_PATH)
    os.makedirs(OUTPUT_PATH)
OUTPUT_FILE_PATH = os.path.join(OUTPUT_PATH, "0.npy")

In [3]:
npy_arr = np.load(NPY_PATH)

In [4]:
npy_arr.shape

(6293, 64, 1024, 4)

In [5]:
N_COLS = npy_arr.shape[2]

In [6]:
start_idx_arr = np.arange(0, N_COLS, int(N_COLS/8))
start_idx_arr.shape

(8,)

In [7]:
N_DEL_COLS = int(DYNAMISM_THRESHOLD * N_COLS)
N_DEL_COLS

512

In [8]:
del_offset = int(N_DEL_COLS / start_idx_arr.shape[0])+1
del_offset

65

In [9]:
end_idx_arr = start_idx_arr + del_offset
end_idx_arr.shape

(8,)

In [10]:
idx_list = []
for start_idx, end_idx in zip(start_idx_arr, end_idx_arr):
    idx_list += list(range(start_idx, end_idx))
len(idx_list)

520

In [11]:
np.abs(N_DEL_COLS-len(idx_list))

8

In [12]:
assert np.abs(N_DEL_COLS-len(idx_list)) <= 10

In [13]:
npy_arr[:,:,idx_list,3] = 1

In [ ]:
np.save(OUTPUT_FILE_PATH, npy_arr)

In [14]:
npy_arr.shape

(6293, 64, 1024, 4)

In [15]:
img_arr = npy_arr[10]
img_arr.shape

(64, 1024, 4)

In [16]:
import open3d as o3d

In [17]:
pts_arr = img_arr[:,:,:3].reshape((-1,3))
pts_arr.shape

(65536, 3)

In [18]:
clr_arr = np.expand_dims(img_arr[:,:,3], axis=-1)
clr_arr.shape

(64, 1024, 1)

In [19]:
clr_arr = np.concatenate((clr_arr, np.zeros(clr_arr.shape), np.zeros(clr_arr.shape)), axis=-1).reshape((-1,3))
clr_arr.shape

(65536, 3)

In [20]:
this_pcd = o3d.geometry.PointCloud()
this_pcd.points = o3d.utility.Vector3dVector(pts_arr)
this_pcd.colors = o3d.utility.Vector3dVector(clr_arr)
this_pcd

geometry::PointCloud with 65536 points.

In [21]:
def draw_pcd(pcd, where='opn_nb'):
    if where is 'opn_nb':
        visualizer = o3d.JVisualizer()
        visualizer.add_geometry(pcd)
        visualizer.show()
    elif where is 'opn_view':
        o3d.visualization.draw_geometries([pcd], width=1280, height=800)
    elif where is 'mat_3d':
        plt.figure()
        pts = np.asarray(pcd.points)
        plt.scatter(pts[:,0], pts[:,1], pts[:,2])
        plt.grid()
        plt.show()
    elif where is 'mat_2d':
        plt.figure()
        pts = np.asarray(pcd.points)
        plt.scatter(pts[:,0], pts[:,1])
        plt.grid()
        plt.show()

In [22]:
draw_pcd(this_pcd)

JVisualizer with 1 geometries